# Imports

In [ ]:
!pip install yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.0/112.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
 

In [ ]:
import yolov5
import torch
import os
import zipfile

os.environ["PYTHONWARNINGS"] = "ignore::FutureWarning"

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import cv2
from pathlib import Path
from tqdm import tqdm
import numpy as np
import shutil
import warnings

warnings.filterwarnings("ignore", message="`torch.cuda.amp.autocast.*`", category=FutureWarning)

# Utils

In [ ]:
def extract_gt_bbox_from_filename(filename):
    parts = filename.split("-")
    if len(parts) < 3:
        raise ValueError(f"Non valid name file: {filename}")

    bbox_str = parts[2]  # esempio: '154&383_386&473'
    try:
        point1_str, point2_str = bbox_str.split("_")
        x1, y1 = map(int, point1_str.split("&"))
        x2, y2 = map(int, point2_str.split("&"))
    except Exception as e:
        raise ValueError(f"Error in retrieving box information from {filename}: {e}")

    return x1, y1, x2, y2


def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea + 1e-6)

# Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
zip_path = '/content/drive/MyDrive/Dataset_CV/DETECTION/CCPD2019_detection.zip'
extract_path = '/content'


os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print('End of processing')

End of processing


# Network (yolov5)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'{device}')

cuda


In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17511, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 17511 (delta 5), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17511/17511), 16.65 MiB | 28.47 MiB/s, done.
Resolving deltas: 100% (11996/11996), done.
/content/yolov5


In [ ]:
!wget https://huggingface.co/keremberke/yolov5s-license-plate/resolve/main/best.pt -O yolov5s-license-plate.pt

--2025-07-05 17:25:30--  https://huggingface.co/keremberke/yolov5s-license-plate/resolve/main/best.pt
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.74, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/06/24/0624a9d885f6bed31eafe60386ae9fe863034b634bd5dc437e72d464d9629761/eabbd7890ffb3d240230510285bb0b5f5c601fec86b1020cfa96f1ff8c9640d6?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27best.pt%3B+filename%3D%22best.pt%22%3B&Expires=1751739930&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MTczOTkzMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8wNi8yNC8wNjI0YTlkODg1ZjZiZWQzMWVhZmU2MDM4NmFlOWZlODYzMDM0YjYzNGJkNWRjNDM3ZTcyZDQ2NGQ5NjI5NzYxL2VhYmJkNzg5MGZmYjNkMjQwMjMwNTEwMjg1YmIwYjVmNWM2MDFmZWM4NmIxMDIwY2ZhOTZmMWZmOGM5NjQwZDY%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1

#  Train

In [ ]:
custom_hyp = """
# --- ottimizzazione standard (copiati da hyp.scratch.yaml) ---
lr0: 0.01
lrf: 0.01
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
warmup_bias_lr: 0.1

# --- bilanciamento perdite ---
box: 0.05
cls: 0.3
cls_pw: 1.0
obj: 0.7
obj_pw: 1.0
iou_t: 0.20
anchor_t: 4.0
fl_gamma: 0.0

# --- augmentation personalizzata ---
hsv_h: 0.05
hsv_s: 0.7
hsv_v: 0.6
degrees: 10.0
translate: 0.1
scale: 0.7
shear: 2.0
perspective: 0.0005
flipud: 0.2
fliplr: 0.5
mosaic: 1.0
mixup: 0.2
copy_paste: 0.0
"""
output_path = "/content/yolov5/data/hyps/hyp.custom.yaml"

with open(output_path, "w") as f:
    f.write(custom_hyp)

In [ ]:
!python train.py \
  --img 640 \
  --batch 50 \
  --epochs 10 \
  --data /content/drive/MyDrive/Computer_vision/ccpd.yaml \
  --weights yolov5s-license-plate.pt \
  --hyp /content/yolov5/data/hyps/hyp.custom.yaml \
  --project /content/drive/MyDrive/Computer_vision/YoloWeights_finetune \
  --name yoloplate_finetuned \
  --save-period 2


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-17 17:04:38.345586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750179878.380094    6281 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750179878.392292    6281 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5m-license-plate.pt, cfg=, data=/content/drive/MyDrive/Com

In [ ]:
# to restart training
!python train.py --resume /content/drive/MyDrive/Computer_vision/YoloWeights_finetune/yoloplate_finetuned/weights/last.pt


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-18 13:08:04.962590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750252085.262292    1632 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750252085.341177    1632 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

# Test

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom',
                       path='/content/drive/MyDrive/Computer_vision/YoloWeights_finetune/yoloplate_finetuned/weights/best.pt')

model.conf = 0.2

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-7-5 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
base_path = Path("/content/CCPD2019_detection/images/test")
accuracy_report = {}

for folder in sorted(base_path.iterdir()):
    if not folder.is_dir():
        continue

    correct = 0
    wrong = 0
    missed = 0
    total = 0

    print(f"\nAnalizing {folder.name}...")

    for img_path in tqdm(list(folder.glob("*.jpg"))):
        img = cv2.imread(str(img_path))
        if img is None:
            continue

        total += 1

        gt_box = extract_gt_bbox_from_filename(img_path.name)
        results = model(img)

        if len(results.pred[0]) == 0:
            missed += 1
            continue

        x1, y1, x2, y2 = map(int, results.pred[0][0][:4])
        pred_box = (x1, y1, x2, y2)

        iou = compute_iou(gt_box, pred_box)
        if iou > 0.7:
            correct += 1
        else:
            wrong += 1

    predicted = correct + wrong
    accuracy_total = correct / total * 100 if total else 0
    accuracy_detect = correct / predicted * 100 if predicted else 0

    accuracy_report[folder.name] = {
        "correct": correct,
        "wrong": wrong,
        "missed": missed,
        "total": total,
        "accuracy_total": round(accuracy_total, 2),
        "accuracy_detect": round(accuracy_detect, 2)
    }

# Print results
print("\n=== Yolo accuracy with (IoU > 0.7) ===")
for folder, stats in accuracy_report.items():
    print(f"{folder}:")
    print(f"   Correct           : {stats['correct']}")
    print(f"   Wrong IoU         : {stats['wrong']}")
    print(f"   No Detection      : {stats['missed']}")
    print(f"   Total Accuracy    : {stats['accuracy_total']}% ({stats['correct']}/{stats['total']})")
    print(f"   Detected Accuracy : {stats['accuracy_detect']}% ({stats['correct']}/{stats['correct'] + stats['wrong']})")



Analizing ccpd_base...


100%|██████████| 10000/10000 [02:19<00:00, 71.44it/s]



Analizing ccpd_blur...


100%|██████████| 10000/10000 [02:08<00:00, 77.58it/s]



Analizing ccpd_challenge...


...


Analizing ccpd_tilt...


100%|██████████| 10000/10000 [02:31<00:00, 66.17it/s]



Analizing ccpd_weather...


100%|██████████| 9999/9999 [02:33<00:00, 64.95it/s]


=== Yolo accuracy with (IoU > 0.7) ===
ccpd_base:
   Correct           : 9074
   Wrong IoU         : 282
   No Detection      : 644
   Total Accuracy    : 90.74% (9074/10000)
   Detected Accuracy : 96.99% (9074/9356)
ccpd_blur:
   Correct           : 3582
   Wrong IoU         : 878
   No Detection      : 5540
   Total Accuracy    : 35.82% (3582/10000)
   Detected Accuracy : 80.31% (3582/4460)
ccpd_challenge:
   Correct           : 6324
   Wrong IoU         : 791
   No Detection      : 2885
   Total Accuracy    : 63.24% (6324/10000)
   Detected Accuracy : 88.88% (6324/7115)
ccpd_db:
   Correct           : 3760
   Wrong IoU         : 1438
   No Detection      : 4802
   Total Accuracy    : 37.6% (3760/10000)
   Detected Accuracy : 72.34% (3760/5198)
ccpd_fn:
   Correct           : 3832
   Wrong IoU         : 767
   No Detection      : 5401
   Total Accuracy    : 38.32% (3832/10000)
   Detected Accuracy : 83.32% (3832/4599)
ccpd_rotate:
   Correct           : 4988
   Wrong IoU         : 6

# Save cropped images

In [ ]:
# Save files with IoU>0.6
base_path = Path("/content/CCPD2019_detection/images/test")

output_dir = Path("/content/detected_images")
if output_dir.exists():
    shutil.rmtree(output_dir)
output_dir.mkdir(parents=True, exist_ok=True)

for folder in sorted(base_path.iterdir()):
    if not folder.is_dir():
        continue

    print(f"\n[Save images from {folder.name}]")

    out_subfolder = output_dir / folder.name
    out_subfolder.mkdir(parents=True, exist_ok=True)

    for img_path in tqdm(list(folder.glob("*.jpg"))):
        img = cv2.imread(str(img_path))
        if img is None:
            continue

        gt_box = extract_gt_bbox_from_filename(img_path.name)
        results = model(img)

        if len(results.pred[0]) == 0:
            continue

        x1, y1, x2, y2 = map(int, results.pred[0][0][:4])
        pred_box = (x1, y1, x2, y2)
        iou = compute_iou(gt_box, pred_box)

        if iou > 0.6:
          x1_crop, y1_crop, x2_crop, y2_crop = pred_box
          cropped_img = img[y1_crop:y2_crop, x1_crop:x2_crop]

          # 48x144
          resized_img = cv2.resize(cropped_img, (144, 48))

          dest_path = out_subfolder / img_path.name
          cv2.imwrite(str(dest_path), resized_img)


# Zip file
zip_path = "/content/drive/MyDrive/Computer_vision/detected_images.zip"
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', output_dir)

print(f"\nImages saved on Drive in: {zip_path}")



[Salvataggio immagini da ccpd_base]


100%|██████████| 10000/10000 [02:31<00:00, 66.00it/s]



[Salvataggio immagini da ccpd_blur]


100%|██████████| 10000/10000 [02:20<00:00, 71.37it/s]



[Salvataggio immagini da ccpd_challenge]


...


[Salvataggio immagini da ccpd_tilt]


100%|██████████| 10000/10000 [02:22<00:00, 70.39it/s]



[Salvataggio immagini da ccpd_weather]


100%|██████████| 9999/9999 [02:28<00:00, 67.47it/s]



✅ Archivio salvato su Drive in: /content/drive/MyDrive/Computer_vision/detected_images.zip
